<a href="https://colab.research.google.com/github/joangog/object-detection/blob/main/coco2017_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Model evaluation (inference) on COCO 2017 dataset

The following models will be evaluated:

| Model | Backbone | Image Size | Parameters | GFLOPS 
| --- | --- | --- | --- | --- |
| SSD300 | VGG16 | 300x300 | 35.6M | 69.8 |
| SSDlite320 | MobileNetV3-Large | 320x320 | 3.4M | 1.2 |
| Faster R-CNN |  ResNet-50 FPN || 41.8M |  |
| Faster R-CNN |  MobileNetV3-Large FPN || 19.4M |  |
| Mask R-CNN |  ResNet-50 FPN || 44.4M | |
| YOLOv5s |  Custom | 640x640 | 7.3M | 17 |
| YOLOv5m |  Custom | 640x640 | 21.4M | 51.3 |
| YOLOv5l |  Custom |640x640 | 47M | 115.5 |
| YOLOv3 |  Darknet53 | 640x640 | 70M | 156.3 |
| YOLOv3-tiny |  Darknet53 | 640x640 | 8.8M | 13.3 |
| YOLOv3-spp |  Darknet53 | 640x640 | 63M | 157.1 |

<br>

**Note: GPU Runtime needed**

*Example experiment: Tesla K80, 460.32.03, 11441 MiB, batch_size=8, workers=2*




In [ ]:
# Show system specs
!nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv

name, driver_version, memory.total [MiB]
Tesla K80, 460.32.03, 11441 MiB


### Get requirements

In [ ]:
%%shell

# Install Yolov5
cd /content
git clone https://github.com/ultralytics/yolov5
cd yolov5
pip install --quiet -r requirements.txt

In [ ]:
%%shell

# Install Yolov3
cd /content
git clone https://github.com/ultralytics/yolov3
cd yolov3
pip install --quiet -r requirements.txt

fatal: destination path 'yolov3' already exists and is not an empty directory.


In [ ]:
%%shell

# Install flops-counter
pip install ptflops

In [ ]:
%%shell

# Clone asset files
cd /content
git clone https://github.com/joangog/object-detection-assets
cd object-detection-assets
mv scripts ../
rm -rf /content/object-detection-assets/

### Import packages

In [ ]:
# Import packages

from google.colab import files
from google.colab import drive
import gdown

import os, sys
import math
import time
import copy

import numpy as np
import pandas as pd
import json
import PIL

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torchvision
import torchvision.models.detection as M
import torchvision.transforms.functional as F
import torchvision.utils as U
from torchvision.datasets import CocoDetection

from ptflops import get_model_complexity_info

import scripts.utils as SU
import scripts.transforms as ST
import scripts.coco_utils as SCU
from scripts.coco_eval import CocoEvaluator

### Define auxiliary functions

In [ ]:
def _get_iou_types(model):
    model_without_ddp = model
    if isinstance(model, torch.nn.parallel.DistributedDataParallel):
        model_without_ddp = model.module
    iou_types = ["bbox"]
    if isinstance(model_without_ddp, M.MaskRCNN):
        iou_types.append("segm")
    if isinstance(model_without_ddp, M.KeypointRCNN):
        iou_types.append("keypoints")
    return iou_types


@torch.no_grad()
def evaluate(model, data_loader, device):
    n_threads = torch.get_num_threads()
    torch.set_num_threads(1)
    cpu_device = torch.device("cpu")
    model.eval()
    metric_logger = SU.MetricLogger(delimiter="  ")
    header = 'Test:'
    coco = SCU.get_coco_api_from_dataset(data_loader.dataset)
    iou_types = _get_iou_types(model)
    coco_evaluator = CocoEvaluator(coco, iou_types)

    # Get label names
    label_ids = data_loader.dataset.coco.getCatIds()
    label_info = data_loader.dataset.coco.loadCats(label_ids)
    label_names = [label['name'] for label in label_info]
    labels = dict(zip(label_ids, label_names))  # Label dictionary with id-name as key-value
    labels_inv = dict(zip(label_names, label_ids))  # Inverse label dictionary with name-id as key-value

    for images, targets in metric_logger.log_every(data_loader, 100, header):

        images = list(img.to(device) for img in images)
        if model.__class__.__name__ == 'AutoShape':  # If model is from YOLOv5 package
            images = [F.to_pil_image(image) for image in images]  # Convert images from tensor to PIL

        if torch.cuda.is_available():
            torch.cuda.synchronize()

        model_time = time.time()
        outputs = model(images)  # Get model predictions
        model_time = time.time() - model_time

        if model.__class__.__name__ == 'AutoShape':  # If model is from YOLO package
            # Format outputs to COCO format
            outputs_formatted = []
            for img_outputs in outputs.xyxy:
                output_bboxes = img_outputs[:, :4]
                output_scores = img_outputs[:, 4]
                output_labels = img_outputs[:, 5].to(cpu_device).apply_(
                    lambda x: labels_inv[label_names[int(x)]])  # Convert YOLO label ids to COCO label ids
                outputs_formatted.append({
                    'boxes': output_bboxes,
                    'scores': output_scores,
                    'labels': output_labels
                })
            outputs = outputs_formatted

        outputs = [{k: v.to(cpu_device) for k, v in t.items()} for t in outputs]

        # test lines #######################

        # label_ids = dataset.coco.getCatIds()
        # label_info = dataset.coco.loadCats(label_ids)
        # label_names = [label['name'] for label in label_info]
        # labels = dict(zip(label_ids,label_names))

        # img = F.convert_image_dtype(images[1],torch.uint8).cpu()

        # true_bboxes = F.Tensor([obj['bbox'] for obj in targets[1]]).cpu()
        # true_labels = [labels[obj['category_id']] for obj in targets[1]]
        # true_img = U.draw_bounding_boxes(img, true_bboxes, true_labels)
        # plt.figure(figsize = (25,7))
        # plt.imshow(F.to_pil_image(true_img))

        # output = outputs[1]
        # pred_bboxes = torch.stack([output['boxes'][i] for i in range(0,len(output['boxes'])) if output['scores'][i] > th])
        # pred_labels_ids = output['labels'].tolist()
        # pred_label_ids = [pred_labels_ids[i] for i in range(0,len(pred_labels_ids)) if output['scores'][i] > th]
        # pred_labels = [labels[label_id] for label_id in pred_label_ids]
        # pred_img = U.draw_bounding_boxes(img, pred_bboxes, pred_labels)
        # plt.figure(figsize = (25,7))
        # plt.imshow(F.to_pil_image(pred_img))

        # fig()

        ###############

        res = {target[0]["image_id"]: output for target, output in zip(targets, outputs) if len(target) != 0}
        evaluator_time = time.time()
        coco_evaluator.update(res)
        evaluator_time = time.time() - evaluator_time
        metric_logger.update(model_time=model_time, evaluator_time=evaluator_time)

    metric_logger.synchronize_between_processes()
    print("Averaged stats:", metric_logger)
    coco_evaluator.synchronize_between_processes()

    coco_evaluator.accumulate()
    coco_evaluator.summarize()
    torch.set_num_threads(n_threads)

    # Model FPS
    batch_size = data_loader.batch_size
    fps = batch_size / metric_logger.meters['model_time'].global_avg

    return coco_evaluator, fps, outputs

def convert_to_xyxy(bboxes):  # formats bboxes from (x,y,w,h) to (x,y,x,y)
  for bbox in bboxes:
    bbox[2] = bbox[0] + bbox[2]
    bbox[3] = bbox[1] + bbox[3]
  return bboxes


### Download COCO 2017 validation dataset

In [ ]:
%%shell

cd /content
mkdir dataset

In [ ]:
%%shell

cd /content/dataset

# Download images
wget 'http://images.cocodataset.org/zips/val2017.zip'
unzip -q 'val2017.zip'
rm 'val2017.zip'

In [ ]:
%%shell

cd /content/dataset

# Download annotations
wget 'http://images.cocodataset.org/annotations/annotations_trainval2017.zip'
unzip -q 'annotations_trainval2017.zip'
rm 'annotations_trainval2017.zip'
cp '/content/dataset/annotations/instances_val2017.json' '/content/dataset/val2017'

### Load COCO 2017 validation dataset

In [ ]:
img_dir = '/content/dataset/val2017'
ann_file = 'instances_val2017.json'  # annotations
ann_path = os.path.join(img_dir,ann_file)

# Define data transforms
transforms = ST.Compose([ST.ToTensor()])

# Create dataset
dataset = CocoDetection(img_dir, ann_path, transforms = transforms)

# Create data loader
batch_size = 8
num_workers = 2
data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, collate_fn=SU.collate_fn)

### Load pre-trained model

In [20]:
%cd /content

# Delete utils package to reload it (if loaded), because YOLOv3 and YOLOv5 have
# the same name for it and it causes error
try:
  sys.modules.pop('utils')
except:
  pass

# @markdown Model Selection { display-mode: 'form', run: 'auto' }
model_name = 'YOLOv3-spp' # @param ['SSD300 VGG16', 'SSDlite320 MobileNetV3-Large', 'Faster R-CNN ResNet-50 FPN', 'Faster R-CNN MobileNetV3-Large FPN', 'Mask R-CNN ResNet-50 FPN', 'YOLOv5s', 'YOLOv5m', 'YOLOv5l', 'YOLOv3', 'YOLOv3-tiny', 'YOLOv3-spp']

# @markdown *Note: If you get the error "Cache may be out of date, try 'force_reload=True'" then restart runtime.*

if model_name == 'SSD300 VGG16':
  model_id = 'ssd300_vgg16'
  model = M.ssd300_vgg16(pretrained=True, progress=True)
  model_img_size = (3,300,300)
elif model_name == 'SSDlite320 MobileNetV3-Large':
  model_id = 'ssdlite320_mobilenet_v3_large'
  model = M.ssdlite320_mobilenet_v3_large(pretrained=True, progress=True)
  model_img_size = (3,320,320)
elif model_name == 'Faster R-CNN ResNet-50 FPN':
  model_id = 'fasterrcnn_resnet50_fpn'
  model = M.fasterrcnn_resnet50_fpn(pretrained=True, progress=True)
  model_img_size = (3,1333,1333) 
elif model_name == 'Faster R-CNN MobileNetV3-Large FPN':
  model_id = 'fasterrcnn_mobilenet_v3_large_fpn'
  model = M.fasterrcnn_mobilenet_v3_large_fpn(pretrained=True, progress=True)
  model_img_size = (3,224,224) 
elif model_name == 'Mask R-CNN ResNet-50 FPN':
  model_id = 'maskrcnn_resnet50_fpn'
  model = M.maskrcnn_resnet50_fpn(pretrained=True, progress=True)
  model_img_size = (3,224,224)
elif model_name == 'YOLOv5s':
  model_id = 'yolov5s'
  model = torch.hub.load('ultralytics/yolov5', 'yolov5s')
  model_img_size = (3,640,640)
elif model_name == 'YOLOv5m':
  model_id = 'yolov5m'
  model = torch.hub.load('ultralytics/yolov5', 'yolov5m')
  model_img_size = (3,640,640)
elif model_name == 'YOLOv5l':
  model_id = 'yolov5l'
  model = torch.hub.load('ultralytics/yolov5', 'yolov5l')
  model_img_size = (3,640,640)
elif model_name == 'YOLOv3':
  model_id = 'yolov3'
  model = torch.hub.load('ultralytics/yolov3', 'yolov3', force_reload=True)
  model_img_size = (3,640,640)
elif model_name == 'YOLOv3-tiny':
  model_id = 'yolov3_tiny'
  model = torch.hub.load('ultralytics/yolov3', 'yolov3_tiny')
  model_img_size = (3,640,640)
elif model_name == 'YOLOv3-spp':
  model_id = 'yolov3_spp'
  model = torch.hub.load('ultralytics/yolov3', 'yolov3_spp')
  model_img_size = (3,640,640)

print('-------------------------------------------------------------------------------------------------------\n')

print(f'Loaded model: {model_name}')
model_params = sum([param.numel() for param in model.parameters()])
print(f'\t- Parameters: {round(model_params / 1000000, 1)}M')
model_macs, _ = get_model_complexity_info(model, model_img_size, as_strings=False, 
                                          print_per_layer_stat=False, verbose=False)
model_flops = 2 * int(model_macs)
print(f'\t- GFLOPS: {round(model_flops / 1000000000, 1)}')

/content


Using cache found in /root/.cache/torch/hub/ultralytics_yolov3_master
Fusing layers... 
Model Summary: 269 layers, 62971933 parameters, 0 gradients
Adding AutoShape... 
YOLOv3 🚀 2021-8-27 torch 1.9.0+cu102 CUDA:0 (Tesla K80, 11441.1875MB)



-------------------------------------------------------------------------------------------------------

Loaded model: YOLOv3-spp
	- Parameters: 63.0M
	- GFLOPS: 157.1


### (Optional) Test model with image sample

In [21]:
# Parameters
img_id = 139
th = 0.5  # threshold for confidence score of predicted bboxes to show

# Get appropriate device for model
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Get image sample
img = PIL.Image.open(os.path.join(img_dir,dataset.coco.loadImgs([img_id])[0]['file_name']))
img_tensor = F.convert_image_dtype(F.to_tensor(img),torch.uint8)
img_torchvision = torch.div(img_tensor,255).float().to(device)  # Format image for torchvision models
img_anns = dataset.coco.loadAnns(dataset.coco.getAnnIds([img_id]))

# Generate model predictions
model.eval()
with torch.no_grad():
  if 'YOLO' in model_name:
    pred = model([img])
  else:    
    pred = model([img_torchvision])

# Get label names
label_ids = dataset.coco.getCatIds()
label_info = dataset.coco.loadCats(label_ids)
label_names = [label['name'] for label in label_info]
labels = dict(zip(label_ids,label_names))  # Label dictionary with id-name as key-value
labels_inv = dict(zip(label_names,label_ids))  # Inverse label dictionary with name-id as key-value

# Get ground truth bboxes
true_bboxes = convert_to_xyxy(copy.deepcopy(F.Tensor([obj['bbox'] for obj in img_anns]).to(device)))  # Create deep copy to avoid updating original dataset
true_labels = [labels[obj['category_id']] for obj in img_anns]
true_img = U.draw_bounding_boxes(img_tensor, true_bboxes, true_labels)
plt.figure(figsize = (25,7))
plt.title('Ground Truth Detection')
plot = plt.imshow(F.to_pil_image(true_img))

# Get ground truth segmentation masks
# print(img_anns)
# true_masks = F.Tensor([obj['segmentation'][0] for obj in img_anns])
# true_labels = [labels[obj['category_id']] for obj in img_anns]
# true_img = U.draw_segmentation_masks(img, true_masks, true_labels)
# plt.figure(figsize = (25,7))
# plt.imshow(F.to_pil_image(true_img))

# Get predicted bboxes
# For YOLO models
if 'YOLO' in model_name:  
  pred_bboxes = []
  pred_labels = []
  for img in pred.xyxy:
    for bbox in img:
      if bbox[4] > th:  # Show only bboxes with high confidence score
        pred_bboxes.append(bbox[:4])
        pred_labels.append(labels[labels_inv[label_names[int(bbox[5])]]])  # Convert YOLO label to COCO label
  pred_bboxes = torch.stack(pred_bboxes)
# For torchvision models
else:
  pred_bboxes = torch.stack([pred[0]['boxes'][i] for i in range(0,len(pred[0]['boxes'])) if pred[0]['scores'][i] > th])  # Show only bboxes with high confidence score
  pred_labels_ids = torch.stack([pred[0]['labels'][i] for i in range(0,len(pred[0]['labels'])) if pred[0]['scores'][i] > th]).tolist()
  pred_labels = [labels[label_id] for label_id in pred_labels_ids]
pred_img = U.draw_bounding_boxes(img_tensor, pred_bboxes, pred_labels)
plt.figure(figsize = (25,7))
plt.title(f'Predicted Detection (thresh={th})')
plot = plt.imshow(F.to_pil_image(pred_img))

### Evaluate model on dataset

In [ ]:
# Get appropriate device for model
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
print(f'Model: {model_name}')

# Evaluate model
evaluator, fps, outputs = evaluate(model, data_loader, device)

Model: YOLOv3-spp
Test:  [  0/625]  eta: 0:14:10  model_time: 0.9027 (0.9027)  evaluator_time: 0.0279 (0.0279)  time: 1.3607  data: 0.3321  max mem: 775
Test:  [100/625]  eta: 0:07:49  model_time: 0.7590 (0.7548)  evaluator_time: 0.0313 (0.0324)  time: 0.8980  data: 0.0170  max mem: 775
Test:  [200/625]  eta: 0:06:20  model_time: 0.7600 (0.7541)  evaluator_time: 0.0350 (0.0353)  time: 0.9046  data: 0.0183  max mem: 775
Test:  [300/625]  eta: 0:04:50  model_time: 0.7637 (0.7547)  evaluator_time: 0.0277 (0.0340)  time: 0.8882  data: 0.0167  max mem: 775
Test:  [400/625]  eta: 0:03:21  model_time: 0.7671 (0.7554)  evaluator_time: 0.0263 (0.0333)  time: 0.9042  data: 0.0161  max mem: 775
Test:  [500/625]  eta: 0:01:51  model_time: 0.7656 (0.7556)  evaluator_time: 0.0248 (0.0342)  time: 0.8806  data: 0.0156  max mem: 775
Test:  [600/625]  eta: 0:00:22  model_time: 0.7657 (0.7557)  evaluator_time: 0.0313 (0.0337)  time: 0.8801  data: 0.0156  max mem: 775
Test:  [624/625]  eta: 0:00:00  model

### Save results

In [ ]:
cols = ['model', 'model_params', 'fps', 'iou_type', 'metric', 'iou', 'area', 'max_dets', 'score']
model = []
model_params_arr = []
fps_arr = []
iou_type = []
metric = []
iou = []
area = []
max_dets = []
score = []

# Set column values
for curr_iou_type, coco_eval in evaluator.coco_eval.items():
  model += [model_name for i in range(0,12)]
  model_params_arr += [model_params for i in range(0,12)]
  fps_arr += [fps for i in range(0,12)]
  iou_type += [curr_iou_type for i in range(0,12)]
  metric += ['avg_precision' for i in range(0,6)] + ['avg_recall' for i in range(0,6)]
  iou += ['0.50:0.95', '0.50 ', '0.75'] + ['0.50:0.95' for i in range(0,9)]
  area += ['all' for i in range(0,3)] + ['small', 'medium', 'large'] + ['all' for i in range(0,3)] + ['small', 'medium', 'large'] 
  max_dets += [100 for i in range(0,6)] + [1, 10] + [100 for i in range(0,4)]
  score += list(coco_eval.stats)

results = pd.DataFrame(np.column_stack([model, model_params_arr, fps_arr, iou_type, metric, iou, area, max_dets, score]))
results.columns = cols

# Safe file
results_file = f'coco17_{model_id}_results.csv'
results_dir = '/content'
results_path = os.path.join(results_dir, results_file)
if os.path.exists(results_path):
      os.remove(results_path)
with open(results_path, 'w') as outfile: 
    results.to_csv(outfile)

In [ ]:
# Save file to GDrive
drive.mount('/content/drive')
gdrive_results_dir = '/content/drive/MyDrive/object-detection-outputs/coco2017'
gdrive_results_path = os.path.join(gdrive_results_dir, results_file)
if os.path.exists(gdrive_results_path):
      os.remove(gdrive_results_path)
with open(gdrive_results_path, 'w') as outfile: 
    results.to_csv(outfile)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# results = []
# for i, img_id in enumerate(img_ids):  # For every image
#   img_predictions = predictions[i]
#   pred_boxes = img_predictions['boxes'].tolist()
#   pred_labels = img_predictions['labels'].tolist()
#   pred_scores = img_predictions['scores'].tolist()
#   for j in range(0,len(pred_box)):  # For every predicted object
#     pred_box = pred_boxes[j]
#     results.append({
#         'image_id': img_id,
#         'category_id': pred_labels[j],
#         'bbox': [round(val,1) for val in pred_box],  # Round for lower file size
#         'score': pred_scores[j]
#         })

# results_file = os.path.join(img_dir,f'{model_id}_coco17_results.json')
# with open(results_file, 'w') as outfile: 
#     json.dump(results, outfile,indent = 2)